# Informe del Proyecto: Clasificación de Imágenes de Espectrogramas con VGG16

## 1. Introducción

El presente cuadernillo tiene como objetivo desarrollar un modelo de aprendizaje profundo para la clasificación binaria de espectrogramas, utilizando la arquitectura preentrenada VGG16. La tarea principal es identificar si una imagen pertenece a una clase específica, a partir de imágenes espectrales almacenadas en un directorio.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau

ImportError: Traceback (most recent call last):
  File "c:\Users\Vicen\anaconda3\envs\tensorflow_env\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: Module use of python38.dll conflicts with this version of Python.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

## 2.Preparación de los Datos

### 2.1 Configuración de Parámetros

Se establecieron los siguientes parámetros para el preprocesamiento y la configuración del modelo:

*    Altura y Ancho de Imágenes: 128x128 píxeles
*    Tamaño de Lote (Batch Size): 32
*    Número de Épocas: 5
*    Directorio Base de Imágenes: Ruta específica donde se encuentran los espectrogramas

In [12]:
def get_parameters():
    params = {
        'img_height': 128,
        'img_width': 128,
        'batch_size': 32,
        'epochs': 5,
        'learning_rate': 0.0001,
        'base_dir': '/home/yvan/Escritorio/Acustic/img/Spectrograms'
    }
    return params

### 2.2 Generación de Datos de Entrenamiento y Validación

Se utilizó ImageDataGenerator para realizar aumentación y normalización de imágenes. Las imágenes fueron divididas en un 80% para entrenamiento y un 20% para validación.

*    Rescale: Las imágenes fueron normalizadas a un rango de 0,10,1.
*    Aumentación: Se implementaron rotaciones, desplazamientos horizontales y verticales, y volteo horizontal.

In [13]:
# Función para crear generadores de datos
def create_data_generators(base_dir, img_height, img_width, batch_size):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )
    validation_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )
    return train_generator, validation_generator

## 3. Diseño del Modelo

### 3.1 Arquitectura del Modelo

La arquitectura del modelo se basa en VGG16 preentrenado en el conjunto de datos ImageNet, con las siguientes modificaciones:

*    Capas Congeladas: Todas las capas de VGG16 se congelaron para evitar la actualización de los pesos preentrenados.
*    Capas Personalizadas: Se añadieron capas de aplanado (Flatten), densa (Dense) y de abandono (Dropout) para mejorar la capacidad de generalización del modelo.

In [14]:
# Función para crear y compilar el modelo
def create_model(img_height, img_width, learning_rate):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    for layer in base_model.layers:
        layer.trainable = False
    
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy', Recall()])
    return model


In [15]:
# Entrenamiento del modelo
def train_model(model, train_generator, validation_generator, epochs):
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[reduce_lr]
    )
    return history


In [16]:
#Ejecución
params = get_parameters()
train_generator, validation_generator = create_data_generators(
    params['base_dir'], params['img_height'], params['img_width'], params['batch_size']
)
model = create_model(params['img_height'], params['img_width'], params['learning_rate'])
history = train_model(model, train_generator, validation_generator, params['epochs'])

Found 3540 images belonging to 2 classes.
Found 884 images belonging to 2 classes.
Epoch 1/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.8559 - loss: 0.3138 - recall_1: 0.9050 - val_accuracy: 0.8891 - val_loss: 0.2026 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 169s 2s/step - accuracy: 1.0000 - loss: 0.0155 - recall_1: 1.0000 - val_accuracy: 0.9400 - val_loss: 0.1154 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 1.0000 - loss: 0.0058 - recall_1: 1.0000 - val_accuracy: 0.9378 - val_loss: 0.1157 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 169s 2s/step - accuracy: 1.0000 - loss: 0.0036 - recall_1: 1.0000 - val_accuracy: 0.9446 - val_loss: 0.1047 - val_recall_1: 1.0000 - learning_rate: 1.0000e-04
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - accuracy: 1.0000 - loss: 0.0023 - recall_1: 1.0000 - val_accuracy: 0.96

## 4. Análisis de los Resultados

El modelo fue entrenado durante 5 épocas, utilizando el generador de datos de entrenamiento y validación. Se empleó el callback ReduceLROnPlateau para reducir la tasa de aprendizaje en caso de estancamiento en la mejora de la función de pérdida.

Se observa una mejora constante en la precisión del conjunto de entrenamiento con cada época, alcanzando un valor máximo del 99.08% en la última época. Sin embargo, la precisión en el conjunto de validación muestra una mejora más gradual y no alcanza valores tan altos, situándose en torno al 84%.

Este comportamiento puede sugerir que el modelo podría estar empezando a sobreajustarse al conjunto de entrenamiento en épocas posteriores. Una pérdida de validación relativamente estable (disminuyendo de 0.3406 a 0.3036) indica que el modelo sigue generalizando adecuadamente, aunque puede ser beneficioso implementar regularización adicional o ajustar parámetros para mejorar la precisión de validación.